In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle
from ili.validation.metrics import PosteriorCoverage
from os.path import join

In [3]:
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor

def process_item(item):
    (i, j), value = item
    x = value['x']
    # Create additional channels filled with i and j
    i_channel = np.full_like(x, i)
    j_channel = np.full_like(x, j)
    # Concatenate the original x with the new channels
    x_with_channels = np.stack((x, i_channel, j_channel))
    
    theta = value['params']
    concatenated_theta = np.concatenate((theta, [i, j]))
    
    return x_with_channels, concatenated_theta

def process_dictionary(data_dict):
    x_list = []
    theta_list = []

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_item, data_dict.items()))

    for x_with_channels, concatenated_theta in results:
        x_list.append(x_with_channels)
        theta_list.append(concatenated_theta)

    x_tensor = torch.tensor(np.array(x_list))
    theta_tensor = torch.tensor(np.array(theta_list))

    return x_tensor, theta_tensor



In [4]:
sigma = 0.0
with open(f'./script/uncertanties_testing/test_set/test_set_{sigma}.pkl', 'rb') as pickle_file:
    data = pickle.load(pickle_file)
x_test, params_test = process_dictionary(data)


posterior_dict = {}
m = 'NPE'
with open(join(f'./script/uncertanties_testing/posterior/posterior_{sigma}', 'posterior.pkl'), 'rb') as f:
    posterior_dict[m] = pickle.load(f)
    
# Assuming posterior_dict['NPE'] is the object containing the given dictionary structure
npe_object = posterior_dict['NPE']

def set_device(module, device):
    """
    Recursively set the device for all submodules that have a 'to' method.
    """
    if hasattr(module, 'to'):
        module.to(device)
    for child in module.children():
        set_device(child, device)

# Access the 'posteriors' ModuleList
posteriors = npe_object._modules['posteriors']

# Set the device for each LampeNPE object and its submodules
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
for i, lampe_npe in enumerate(posteriors):
    set_device(lampe_npe, device)
    embedding_net = lampe_npe.embedding_net
    print(f"Embedding Net for LampeNPE {i} is now on device: {device}")

# Optionally, set the device for the main NPE object itself
set_device(npe_object, device)
    

Embedding Net for LampeNPE 0 is now on device: cuda:6
Embedding Net for LampeNPE 1 is now on device: cuda:6
Embedding Net for LampeNPE 2 is now on device: cuda:6
Embedding Net for LampeNPE 3 is now on device: cuda:6


In [30]:
plot_hist = ["predictions"]
metric = PosteriorCoverage(
    num_samples=2_001, sample_method='direct',
    labels=[rf'$\log_{{10}}(M_{{s}}) [M_{{\odot}}]$', rf'$\log_{{10}}(\tau) [Gyr]$'], plot_list = plot_hist
)

fig = metric(
    posterior=posterior_dict[m],
    x=x_test, theta=params_test[:, :2])

  0%|          | 8/6515 [00:02<38:23,  2.83it/s]


KeyboardInterrupt: 

In [33]:
posterior_dict[m].sample((1001, ), x=x_test, ).shape

INFO:root:Sampling models with tensor([220., 249., 186., 346.], device='cuda:6') samples each.


Drawing 220 posterior samples: 1430801it [00:06, 212229.91it/s]       
Drawing 249 posterior samples: 1616580it [00:07, 212824.05it/s]       
Drawing 186 posterior samples: 1207832it [00:05, 213038.63it/s]       
Drawing 346 posterior samples: 2252423it [00:09, 241014.16it/s]       


torch.Size([1001, 2])

In [ ]:
posterior_dict = {}
m = 'NPE'
with open(join(f'./script/uncertanties_testing/posterior/posterior_{sigma}', 'posterior.pkl'), 'rb') as f:
    posterior_dict[m] = pickle.load(f)
    
# Assuming posterior_dict['NPE'] is the object containing the given dictionary structure
npe_object = posterior_dict['NPE']

def set_device(module, device):
    """
    Recursively set the device for all submodules that have a 'to' method.
    """
    if hasattr(module, 'to'):
        module.to(device)
    for child in module.children():
        set_device(child, device)

# Access the 'posteriors' ModuleList
posteriors = npe_object._modules['posteriors']

# Set the device for each LampeNPE object and its submodules
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
for i, lampe_npe in enumerate(posteriors):
    set_device(lampe_npe, device)
    embedding_net = lampe_npe.embedding_net
    print(f"Embedding Net for LampeNPE {i} is now on device: {device}")

# Optionally, set the device for the main NPE object itself
set_device(npe_object, device)

In [37]:
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor

def process_item(item):
    (i, j), value = item
    x = value['x']
    # Create additional channels filled with i and j
    i_channel = np.full_like(x, i)
    j_channel = np.full_like(x, j)
    # Concatenate the original x with the new channels
    x_with_channels = np.stack((x, i_channel, j_channel))
    
    theta = value['params']
    concatenated_theta = np.concatenate((theta, [i, j]))
    
    return x_with_channels, concatenated_theta

def process_dictionary(data_dict):
    x_list = []
    theta_list = []
    
    mae_list = []

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_item, data_dict.items()))

    for x_with_channels, concatenated_theta in results:
        sampled_theta = posterior_dict[m].sample((1001, ), x=torch.tensor(x_with_channels))[:, 0] 
        mae_list.append(  np.mean(sampled_theta.cpu().numpy(), axis=0) - concatenated_theta[0] )

    return np.mean(mae_list)


sigma = 0.0
with open(f'./script/uncertanties_testing/test_set/test_set_{sigma}.pkl', 'rb') as pickle_file:
    data = pickle.load(pickle_file)
mae = process_dictionary(data)


INFO:root:Sampling models with tensor([220., 249., 186., 346.], device='cuda:6') samples each.
Drawing 220 posterior samples: 439it [00:00, 3455.74it/s]             
Drawing 249 posterior samples: 100%|██████████| 249/249 [00:00<00:00, 4044.07it/s]
Drawing 186 posterior samples: 370it [00:00, 2748.55it/s]             
Drawing 346 posterior samples: 100%|██████████| 346/346 [00:00<00:00, 5606.08it/s]
INFO:root:Sampling models with tensor([220., 249., 186., 346.], device='cuda:6') samples each.
Drawing 346 posterior samples: 100%|██████████| 346/346 [00:00<00:00, 4844.21it/s]
INFO:root:Sampling models with tensor([220., 249., 186., 346.], device='cuda:6') samples each.
Drawing 346 posterior samples: 100%|██████████| 346/346 [00:00<00:00, 5845.23it/s]
INFO:root:Sampling models with tensor([220., 249., 186., 346.], device='cuda:6') samples each.
Drawing 220 posterior samples: 439it [00:00, 3409.29it/s]             
Drawing 346 posterior samples: 100%|██████████| 346/346 [00:00<00:00, 5711.